# Heuristic Agent

In [1]:
import pandas as pd
import os
import random
import sys

from animalai.envs.environment import AnimalAIEnvironment

import sys
sys.path.append("..")

from aaisrc.aaiyaml import find_yaml_files
from aaisrc.aaiyaml import yaml_combinor
from aaisrc.aaiagents import buttonHeuristic
from aaisrc.aaiagents import Heuristic

In [ ]:
def evaluate_heuristic(env_path : str, configuration_file : str, save_path : str, agent, watch : bool = False, num_runs : int = 100, verbose : bool = True):

        episode_rewards = []

        
        port = 4000 + random.randint(
            0, 1000
            )  # use a random port to avoid problems if a previous version exits slowly
        
        aai_env = AnimalAIEnvironment( 
            inference=watch, #Set true when watching the agent
            seed = 2023,
            worker_id=random.randint(0, 65500),
            file_name=env_path,
            arenas_configurations=configuration_file,
            base_port=port,
            useCamera=False,
            useRayCasts=True,
            raysPerSide = agent.num_rays,
            rayMaxDegrees = agent.max_degrees,
            no_graphics=True,
            timescale=1
        )

        behavior = list(aai_env.behavior_specs.keys())[0] # by default should be AnimalAI?team=0

        random.seed(2023)
        
        for _episode in range(num_runs): 
            if verbose:
                print(f"Running episode {_episode+1} of {num_runs}.") 
            done = False
            episodeReward = 0

            aai_env.step() # take first step to get an observation

            dec, term = aai_env.get_steps(behavior)
        
            while not done:

                observations = aai_env.get_obs_dict(dec.obs)

                action = agent.get_action(observations)

                aai_env.set_actions(behavior, action)

                aai_env.step()

                dec, term = aai_env.get_steps(behavior)
                
                if len(dec.reward) > 0 and len(term) <= 0:
                    episodeReward += dec.reward

                elif len(term) > 0: #Episode is over
                    episodeReward += term.reward
                    print(f"Episode Reward: {episodeReward}")
                    done = True
                    episode_rewards.append(episodeReward)
                
                else:
                    pass

        aai_env.close()

        results_dataframe = pd.DataFrame({"EpisodeNumber" : [x for x in range(num_runs)],
                                     "FinalReward" : episode_rewards})
    
        results_dataframe.to_csv(save_path,
                            index = False)

        return results_dataframe

In [ ]:
def evaluate_braitenbergagent(env_path : str, configuration_folder : str, tmp_file_location : str, save_path_csv : str, watch : bool = False, seed : int = 2023, batch_size: int = 50, port_base : int = 6600):

    rays = 15
    degrees = 30

    braitenberg = Braitenberg(no_rays=rays, max_degrees=degrees, verbose = False)
    
    yaml_index = 0

    port = port_base + yaml_index
        
    batch_counter = 0

    paths, names = find_yaml_files(configuration_folder)

    
    for yaml_index in range(0, len(paths), batch_size):

        completed = False

        while not completed:
            try:
                if (yaml_index + batch_size) > len(paths) or batch_size > len(paths):
                    upper_bound = len(paths)
                else:
                    upper_bound = ((yaml_index + batch_size))
                
                print(f"Running inferences on batch {batch_counter + 1} of {batch_size} files of total {len(paths)}. {len(paths) - (batch_size * (batch_counter + 1))} instances to go.")

                batch_files = paths[yaml_index:upper_bound]

                batch_file_names = names[yaml_index:upper_bound]

                batch_temp_file_name = f"TempConfig_Braitenberg_{seed}_{yaml_index}.yml"

                config_file_path = yaml_combinor(file_list = batch_files, temp_file_location=tmp_file_location, stored_file_name = batch_temp_file_name)
            
                aai_env = AnimalAIEnvironment( 
                    inference=watch, #Set true when watching the agent
                    seed = 2023,
                    worker_id=random.randint(0, 65500),
                    file_name=env_path,
                    arenas_configurations=config_file_path,
                    base_port=port,
                    useCamera=False,
                    useRayCasts=True,
                    raysPerSide = rays,
                    rayMaxDegrees = degrees,
                    no_graphics=(not watch),
                    timescale=1
                )

                behavior = list(aai_env.behavior_specs.keys())[0] # by default should be AnimalAI?team=0

                random.seed(seed)
                
                for _episode in range(batch_size): 
                    print(f"Running episode {_episode+1} of {batch_size}.") 

                    done = False
                    episodeReward = 0

                    aai_env.step() # take first step to get an observation

                    dec, term = aai_env.get_steps(behavior)
                
                    while not done:

                        observations = aai_env.get_obs_dict(dec.obs)

                        action = braitenberg.get_action(observations)

                        aai_env.set_actions(behavior, action)

                        aai_env.step()

                        dec, term = aai_env.get_steps(behavior)
                        
                        if len(dec.reward) > 0 and len(term) <= 0:
                            episodeReward += dec.reward

                        elif len(term) > 0: #Episode is over
                            episodeReward += term.reward
                            print(f"Episode Reward: {episodeReward}")
                            done = True
                        
                        else:
                            pass
                
                    file_exists = os.path.isfile(save_path_csv)
                    with open(save_path_csv, 'a' if file_exists else 'w', newline='') as csv_file:
                        csv_write = csv.writer(csv_file)
                        if not file_exists:
                            csv_write.writerow(['episode', 'finalReward'])
                        csv_write.writerow([str(names[yaml_index+_episode]), str(episodeReward)])
                    print(f"Writing episode score {episodeReward} for episode {names[yaml_index+_episode]} to {save_path_csv}")

                aai_env.close()
                os.remove(config_file_path)
                batch_counter += 1
                completed = True
            except:
                print("Episode failed. Retrying")
                port += 1
                try: 
                    aai_env.close()
                except:
                    pass
                completed = False
                break
    
    return True

In [ ]:
foraging_results = evaluate_heuristic(env_path = "../env/AnimalAI.exe",
                                       configuration_file = "../configs/foragingTask/foragingTaskSpawnerTree.yml",
                                       save_path = "../analysis/data/foraging/heuristicAgent100Foraging.csv",
                                       agent = Heuristic(no_rays = 15, max_degrees=30),
                                       num_runs=100)

In [ ]:
button_results = evaluate_heuristic(env_path = "../env/AnimalAI.exe",
                                       configuration_file = "../configs/foragingTask/foragingTaskSpawnerTree.yml",
                                       save_path = "../analysis/data/foraging/heuristicAgent100Button.csv",
                                       agent = buttonHeuristic(no_rays = 101, max_degrees=179),
                                       num_runs=100)

In [ ]:
seeds = [1357, 1602, 1319, 5281, 8198]
for seed in seeds:
    save_path = f"../analysis/data/competitionAAITestbed/Heuristic_{seed}.csv"
    eval = evaluate_braitenbergagent(env_path="../env/AnimalAI.exe",
                                      configuration_folder="../configs/competition",
                                      tmp_file_location="../..",
                                      save_path_csv=save_path,
                                      seed = seed,
                                      port_base = seed + random.randint(0, 1000),
                                      batch_size=1)